In [1]:
import json
import os
from operator import itemgetter
# from attrdict import AttrDict
from pprint import pprint
from IPython.core.debugger import Tracer

In [2]:
workspaceDir = '/home/hanwen/disk/workspace'
dataset = 'test'

In [3]:
# textFile = '{workspaceDir}/../data/{dataset}/merged.txt_without_sentence_id'.format(
#     workspaceDir=workspaceDir, method=method, dataset=dataset)
# inFile = '{workspaceDir}/{method}/result/{dataset}/merged.txt_without_sentence_id.json'.format(
#     workspaceDir=workspaceDir, method=method, dataset=dataset)

In [4]:
# # read json span data and text data
# data = []
# texts = []
# with open(inFile) as fin, open(textFile) as f_text:
#     cnt = 0
#     for line in fin:
#         data.append(json.loads(line))
#         cnt += 1
#         if cnt > 100:
#             break
#     cnt = 0
#     for line in f_text:
#         texts.append(line)
#         cnt += 1
#         if cnt > 100:
#             break        

In [5]:
def read_span_json(inFile, num=None):
    with open(inFile) as fin:
        data = []
        if num:
            for i, line in enumerate(fin):
                data.append(json.loads(line))
                if i >= num:
                    break
        else:
            for line in fin:
                data.append(json.loads(line))
        return data

In [6]:
def read_text(inFile, num=None):
    texts = []
    with open(inFile) as fin, open(textFile) as f_text:
        if num:
            for cnt, line in enumerate(f_text):
                texts.append(line)
                if cnt > num:
                    break
        else:
            for line in f_text:
                texts.append(line)
        return texts

In [7]:
def get_line_count(inFile):
    count = -1
    for count, line in enumerate(open(inFile, 'r')):
        pass
    count += 1
    return count

In [8]:
def check_span_data(inFile_list):
    for inFile in inFile_list:
        print(inFile, 'line count', get_line_count(inFile))

In [9]:
# seginFile = '{}/seg_with_vocab/result/{}/merged.txt_without_sentence_id.json'.format(workspaceDir, dataset)
textFile = '{}/../data/{}/merged.txt_without_sentence_id'.format(workspaceDir, dataset)
nltkFile = '{}/StructMineDataPipeline/result/{}/merged.txt_without_sentence_id.json'.format(workspaceDir, dataset)
spacyFile = '{}/spacy/result/{}/merged.txt_without_sentence_id.json'.format(workspaceDir, dataset)
autoFile = '{}/auto/result/{}/merged.txt_without_sentence_id.json'.format(workspaceDir, dataset)
textrankFile = '{}/textrank/result/{}/merged.txt_without_sentence_id.json'.format(workspaceDir, dataset)
rakeFile = '{}/rake/result/{}/merged.txt_without_sentence_id.json'.format(workspaceDir, dataset)
keaFile = '{}/kea/result/{}/merged.txt_without_sentence_id.json'.format(workspaceDir, dataset)

In [10]:
# check_span_data([textFile, nltkFile, spacyFile, autoFile, textrankFile])

In [11]:
nltkData = read_span_json(nltkFile)
spacyData = read_span_json(spacyFile)
autoData = read_span_json(autoFile)
textrankData = read_span_json(textrankFile)

In [12]:
texts = read_text(textFile)

In [13]:
def merge_span_data(data_list):
    # check data length consistency
    length_list = [len(data) for data in data_list]
    assert len(set(length_list)) == 1
    length = length_list[0]
    new_data = []
    for i in range(length):
        new_d = []
        for data in data_list:
            new_d.extend(data[i])
        new_data.append(new_d)
    return new_data

In [14]:
spacyData[0]

{'entity': [],
 'np': [{'ed': 2, 'st': 0, 'text': 'Semisupervised Clustering'},
  {'ed': 4, 'st': 3, 'text': 'AND-Queries'},
  {'ed': 9, 'st': 5, 'text': 'Locally Encodable Source Coding'}]}

In [15]:
# convert spacy data
spacyNPData = []
spacyEntityData = []
for d in spacyData:
    spacyNPData.append(d['np'])
    spacyEntityData.append(d['entity'])

In [16]:
new_data = merge_span_data([nltkData, spacyNPData, spacyEntityData, autoData, textrankData])

In [17]:
new_data[0]

[{'ed': 2, 'np': [['Clustering', 'NNP']], 'st': 1, 'text': 'Clustering'},
 {'ed': 4, 'np': [['AND-Queries', 'NNP']], 'st': 3, 'text': 'AND-Queries'},
 {'ed': 9,
  'np': [['Locally', 'NNP'],
   ['Encodable', 'NNP'],
   ['Source', 'NNP'],
   ['Coding', 'NNP']],
  'st': 5,
  'text': 'Locally Encodable Source Coding'},
 {'ed': 2, 'st': 0, 'text': 'Semisupervised Clustering'},
 {'ed': 4, 'st': 3, 'text': 'AND-Queries'},
 {'ed': 9, 'st': 5, 'text': 'Locally Encodable Source Coding'},
 {'ed': 7, 'st': 5, 'text': 'Locally Encodable'},
 {'ed': 9, 'st': 7, 'text': 'Source Coding'},
 {'ed': 9, 'st': 7, 'text': 'Source Coding'}]

In [18]:
# def generate_superspan(d, tokens):
#     #     Tracer()()
#     cur_super_st = -1
#     cur_super_ed = -1

#     superspan_list = []
#     for span in sorted(d, key=itemgetter('st', 'ed')):
#         st = span['st']
#         ed = span['ed']
#         text = span['text']
#         if st >= cur_super_ed:
#             # meet a new superspan
# #             if superspan:
# #                 superspan['text'] = ' '.join(
# #                     t.split(' ')[cur_super_st:cur_super_ed])
# #             cur_super_st = st
# #             cur_super_ed = ed
#             superspan = {'super_st': cur_super_st,
#                          'super_ed': cur_super_ed, 'spans': []}
#             if span not in superspan['spans']:
#                 superspan['spans'].append(span)
#             superspan['text'] = ' '.join(tokens)[cur_super_st:cur_super_ed]
#             superspan['tag'] = 'superspan'
#             superspan_list.append(superspan)
#         else:
#             # update super end pos
#             if ed > cur_super_ed:
#                 cur_super_ed = ed
#             superspan['super_ed'] = cur_super_ed
#             superspan['spans'].append({'st': st, 'ed': ed, 'text': text})

#     return superspan_list

In [19]:
def generate_superspan(d, tokens):
    cur_super_st = -1
    cur_super_ed = -1
    superspan_cnt = -1
    superspan = {}
    superspan_list = []
    for span in sorted(d, key=itemgetter('st', 'ed')):
        # span information
        st = span['st']
        ed = span['ed']
        text = span['text']
#         print('text', text)
        
        if st >= cur_super_ed:
            # meet a new superspan
            cur_super_st = st
            cur_super_ed = ed
            superspan = {'super_st':cur_super_st, 'super_ed':cur_super_ed, 'spans':[]}
            if {'st':st, 'ed':ed, 'text':text} not in superspan['spans']:
                superspan['spans'].append({'st':st, 'ed':ed, 'text':text})
                
#             superspan['text'] = ' '.join(tokens[cur_super_st:cur_super_ed])
            superspan['tag'] = 'superspan'
            superspan_list.append(superspan)
            superspan_cnt += 1
        else:
            # update super end pos 
            if ed > cur_super_ed:
                cur_super_ed = ed
            superspan_list[superspan_cnt]['super_ed'] = cur_super_ed
            if {'st':st, 'ed':ed, 'text':text} not in superspan_list[superspan_cnt]['spans']:
                superspan_list[superspan_cnt]['spans'].append({'st':st, 'ed':ed, 'text':text})
#             superspan_list[superspan_cnt]['text'] = ' '.join(tokens[cur_super_st:cur_super_ed])
    for i, superspan in enumerate(superspan_list):
        superspan_list[i]['text'] = ' '.join(tokens[superspan['super_st']:superspan['super_ed']])
    return superspan_list


In [20]:
def remove_duplicate(x):
    # 0 is flag for plain text word
    occur = set()
    return_list = []
    for i in x:
        if i ==0 or i not in occur:
            return_list.append(i)
            occur.add(i)
    return return_list

flag = [0, 1, 1, 0, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 10, 11, 11, 12, 12, 12, 12, 0]
remove_duplicate(flag)

[0, 1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0]

In [21]:
def generate_sequence(superspan_list, tokens):
    '''
        generate super span sequence
    '''
    flag = [0]*len(tokens)

    sequence = []    
    for i, superspan in enumerate(superspan_list):
        st = superspan['super_st']
        ed = superspan['super_ed']
        for idx in range(st, ed):
            flag[idx] = i
   
    flag = remove_duplicate(flag)

    for idx, v in enumerate(flag):
        if v == 0:
            sequence.append({'tag':'plain', 'text':tokens[idx]})
        else:
#             superspan_list[v].update({'tag':'superspan'})
            sequence.append(superspan_list[v])
    return sequence

In [22]:
lineCount = get_line_count(textFile)
lineCount = 20

outDir = '{}/merge_span/result/{}'.format(workspaceDir, dataset)
outListFile = '{}/superspan_list.json'.format(outDir)
outFile = '{}/superspan_sequence.json'.format(outDir)

if not os.path.exists(outDir):
    os.makedirs(outDir)

with open(outListFile, 'w') as fout_list, open(outFile, 'w') as fout:
    for slice_num in range(lineCount):
        text = texts[slice_num].strip()
        if text:
            tokens = text.split(' ')
        else:
            tokens = []
    #     print('slice_num', slice_num)
    #     print('tokens', tokens)
    #     print('data', new_data[slice_num])
        superspan_list = generate_superspan(new_data[slice_num], tokens)


        fout_list.write(json.dumps(superspan_list));
        fout_list.write('\n');

        superspan_sequence = generate_sequence(superspan_list, tokens)

        fout.write(json.dumps(superspan_sequence));
        fout.write('\n');

    #     print(superspan_list)
    #     for superspan in superspan_list:
    #         pprint(superspan)
    #         print('--'*20)
    

591

1

540

1

2

...

1

755

1

943

1

In [46]:
# generate_sequence(superspan_list, tokens)

In [48]:
# superspan_sequence